# Lecture 3: Chains

- The most important building block of `langchain` is **chain**

- A chain combines an **LLM** with a **prompt**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('../Data.csv')

In [ ]:

df.Product[0]
# 'Queen Size Sheet Set'

df.Review[0]

# 'I ordered a king size set. My only criticism would be that I \
# wish seller would offer the king size set with 4 pillowcases. \
# I separately ordered a two pack of pillowcases so I could \
# have a total of four. When I saw the two packages, it looked \
# like the color did not exactly match. Customer service was \
# excellent about sending me two more pillowcases so I would \
# have four that matched. Excellent! For the cost of these sheets, \
# I am satisfied with the characteristics and coolness of the sheets.'

## 1. LLMChain

### Example 1.1 A simple prompt using `LLMChain`

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product)

# 'Royal Bedding Co.'

In [ ]:
product = "blow average pancakes with dull colors"
chain.run(product)

# 'Bland Cakes Inc.'

## 2. Sequential Chain

- **Sequential Chain** runs sequence of chains one after the other

![Sequential Chain](../images/L3/sequential_chain.png)

### 2.1 SimpleSequentialChain

![Simple sequential chain](../images/L3/simple_sequential_chain.png)

### Example 2.1.1: Combining Two `LLM` chains using `SimpleSequentialChain`

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

# > Entering new SimpleSequentialChain chain...
# "Pancake Mediocre Co."
# "Pancake Mediocre Co." is a breakfast chain that serves average quality pancakes \
# to customers in various locations.

# > Finished chain.
# '"Pancake Mediocre Co." is a breakfast chain that serves average quality pancakes \
# to customers in various locations.'

## 2.2. Regular SequentialChain

- `SimpleSequentialChain` works well when there is **single input** and **single output**

- When there are **multiple inputs** and **multiple outputs** we need a `SequentialChain`

![Regular sequential chain](../images/L3/regular_sequential_chain.png)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)

# > Entering new SequentialChain chain...
#
# > Finished chain.
# {'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les \
# mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
# 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the \
# same ones in the store and the taste is much better... Old stock or counterfeit!?",
# 'summary': 'The reviewer finds the taste of the product mediocre and suggests that the \
# product purchased online may be old or counterfeit.',
# 'followup_message': "Réponse : Le critique trouve le goût du produit moyen et suggère que \
# le produit acheté en ligne peut être ancien ou contrefait. Il est important de prendre en \
# compte cet avis car l'achat de produits en ligne peut parfois entraîner des problèmes de \
# qualité. Nous vous recommandons de vérifier la date d'expiration et la source d'achat avant \
# de commander des produits en ligne. Merci pour votre avis constructif."}

## 3. Router Chain

- Routes the input to a subchain based on the information extracted form input

![Router Chain](../images/L3/router_chain.png)

### Example 3.1: A chain which routes input based on the **subject** inferred from input text

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

- A `MultiPromptChain` is a specific type of chain that is used
  when routing between multiple different prompt templates.

In [ ]:
from langchain.chains.router import MultiPromptChain

- `LLMRouterChain` uses an **LLM** to route between subchains

- In this example the **LLM** uses the description and name to
  route between the subchains

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain

- `RouterOutputParser` parses the **LLM** output into a dictionary that can be used
   to determine which subchain to use and what's the input to that chain

In [ ]:
from langchain.chains.router.llm_router import RouterOutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

- This template is used by the **LLM** to route between different subchains.

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not \
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain,
                         verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("What is post quantum cryptography")

In [ ]:
chain.run("Who is the wife of Narendra Modi?")

In [ ]:
chain.run("Who is Jashodaben and what is her connection to Narendra Modi?")

In [ ]:
chain.run("""
          State a nontrivial theorem in algebraic geometry? \
          Also cite one of the sources you consulted for \
          getting information about that.
          """)